<div style="background-color:#f0f8ff; border-radius:8px; padding:12px; text-align:center;">

# **PRACTICA 1: KNN y selección de atributos**

</div>

*Aprendizaje Automático*

---

**Grupo:** G-7312  
**Número de pareja:** 01  
**Miembros:**  
- Leire Bernárdez Vázquez  
- Carmen Reiné Rueda


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold

### **1. Implementación de kNN**

---

***a) Descargue de los datos sobre cáncer de mama***

In [ ]:
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()

***b) Preprocese el dataset siguiendo estos pasos:***

***1. Separe los atributos de las etiquetas***

***2. Divida los datos en una partici ́on con el 70 % de los puntos para training y el 30 % de los puntos para test***

***3. Normalice los datos***

In [ ]:
X = data.data      # atributos
y = data.target    # etiquetas

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

***c) Si hubiera datos ausentes (missing values) y  ́estos se completaran:***


***1. ¿Cómo cree que influiría el orden en el que se realizan las operaciones de normalizar y completar?***

***2. ¿Qué pasaría si primero se completan los datos ausentes y luego se normaliza?***

***3. ¿Y si primero se normaliza y luego se realiza la partición training-test?***

Respuesta:

1. Primero completar y luego normalizar sería lo correcto. Primero imputamos valores faltantes (media, mediana, etc.), y después normalizamos, así la media/desviación se calcula sobre datos coherentes.
2. Primero normalizar y luego completar no sería correcto. Los valores “NaN” distorsionarían la media/desviación, y la normalización no tendría sentido.
3. Normalizar antes de dividir en train/test sería incorrecto. La media y desviación estarían influenciadas por el test set (data leakage). Siempre normalizamos después de dividir.

***d) A continuación, complete la clase KNNClassifier, cuyos atributos son el número de vecinos y una funci ́on distancia (una función cuyas entradas son dos vectores de la misma dimensión, y cuya salida es un numero real positivo). Complete el constructor y los m ́etodos fit y predict.***

    Hecho, resivar el test y añadir cosas para ver si es óptimo

***e) Utilice la clase anterior para predecir las etiquetas de los datos de test, con un número de vecinos k, fijo pero arbitrario.***

In [ ]:
knn = KNNClassifier(k=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

    Aquí hay que usar random????

***f) Responda a la siguiente pregunta: ¿Qué ocurrir ́ıa si hubiera un desbalanceo de clases en el conjunto de entrenamiento? Si esto supone un problema, ¿podría proporcionar una solución?***

Si una clase tiene muchos más ejemplos, KNN tenderá a predecir siempre esa clase.
Soluciones posibles:
- Ponderar los vecinos por la inversa de la distancia.
- Usar técnicas de balanceo (oversampling/undersampling, SMOTE).
- Elegir una métrica de evaluación robusta al desbalance (f1-score, balanced accuracy).

***g) Responda a la siguiente pregunta: ¿Cuál es el coste en memoria del algoritmo KNN? ¿Se le ocurre alguna forma de reducirlo?***

KNN es un algoritmo no paramétrico: guarda todos los datos de entrenamiento en memoria, que implica un coste O(Nxd).
Para reducirlo:

- Usar estructuras de búsqueda (KD-trees, ball trees).
- Usar reducción de dimensionalidad (PCA, selección de atributos).
- Usar un subconjunto representativo de datos (prototyping).

***h) Utilice la funci ́on KNeighborsClassifier de la biblioteca de sklearn y, para el mismo n ́umero de vecinos k prediga las etiquetas del conjunto de test ypredsk. Si ypredcustom son las prediciones de su modelo, ¿cuál es el error medio entre las predicciones ypredsk e ypredcustom? ¿Por qué?***

In [ ]:
sk_knn = KNeighborsClassifier(n_neighbors=5)
sk_knn.fit(X_train, y_train)
y_pred_sk = sk_knn.predict(X_test)

# Error medio entre predicciones
error = (y_pred_sk != y_pred).mean()
print("Error medio:", error)

    Hay que ver si da bien el error y justificar el porqué

***e) Se encuentra usted a un individuo que afirma que, en vez de utilizar KNN, el prefiere usar la siguiente alternativa. Para predecir la etiqueta del punto x, toma los tres puntos más cercanos a x en el conjunto de training.***

***Si las distancias de estos tres puntos al punto x son d1, d2 y d3, y sus respectivas etiquetas son y1, y2 e y3, la predicción vendrá dada por:***

***f (x) = sign ( 3∑ i=1 (yi/di) )***

***¿Considera que este método es mejor que KNN con k = 3? ¿Por qué?***

Dado el modelo propuesto:

- **KNN con k=3 estándar**: vota por mayoría simple. Todos los vecinos cuentan igual, aunque uno esté muy cerca y otro bastante más lejos.
- **Método propuesto**: da más importancia a los vecinos más cercanos. Si un vecino está mucho más cerca que los otros dos, su etiqueta tiene más peso en la decisión.

Teniendo en ceuenta el modelo descrito, podemos sacar las siguientes ventajas y desventajas:

- **Ventaja**: puede ser más robusto en situaciones donde los vecinos están a distancias muy diferentes. Ejemplo: si tienes un vecino muy cercano de clase A y dos vecinos algo más lejanos de clase B, el método ponderado asignará A, mientras que KNN (k=3) daría B.
- **Inconveniente**: si las distancias son muy similares, el método propuesto se comporta casi igual que KNN, pero es más sensible a la escala de las distancias (si no se normalizan bien los atributos, el peso puede distorsionarse).

En conclusión, el método ponderado por distancias suele considerarse una mejora sobre KNN clásico porque aprovecha la información de cuán cerca está cada vecino, no solo cuántos hay. Sin embargo, no siempre será mejor en la práctica, depende de cómo estén distribuidos los datos y de la escala de los atributos.

### **2. Optimización de kNN**

---

***a) Encuentre, utilizando validación cruzada, el número de vecinos óptimos, kopt.***

In [ ]:
# Definir el clasificador base
knn = KNeighborsClassifier()

# Valores de k a probar
param_grid = {'n_neighbors': list(range(1, 31))}

# Validación cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(knn, param_grid, cv=cv, scoring='accuracy')
grid.fit(X_train, y_train)

k_opt = grid.best_params_['n_neighbors']
print("k óptimo:", k_opt)

    Revisar antes de preguntar si lo de param_grid sirve para usarse aquí

***b) Dé la métrica de accuracy sobre el conjunto de test del clasificador KNN usando el valor kopt obtenido.***

In [ ]:
best_knn = KNeighborsClassifier(n_neighbors=k_opt)
best_knn.fit(X_train, y_train)
test_acc = best_knn.score(X_test, y_test)
print("Accuracy en test:", test_acc)

    Para que vaya este primero hay que revisar que el a esté bien (aún así funciona)

***c) ¿Cree que el valor kopt encontrado es el que proporciona mejor accuracy en el conjunto de test?***

No necesariamente, ya que se elige para maximizar el rendimiento en validación cruzada, no en el test. Puede pasar que otro k dé un poco más de accuracy en el test, pero eso sería un sobreajuste al test si lo eligiéramos así. Lo correcto es quedarse con kopt por validación.

***d) Muestre en una gráfica el accuracy frente al n ́umero de vecinos, tanto para el conjunto de training como para el de test.***

In [ ]:
k_values = range(1, 31)
acc_train, acc_test = [], []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    acc_train.append(knn.score(X_train, y_train))
    acc_test.append(knn.score(X_test, y_test))

plt.plot(k_values, acc_train, label="Train")
plt.plot(k_values, acc_test, label="Test")
plt.xlabel("Número de vecinos (k)")
plt.ylabel("Accuracy")
plt.title("Accuracy vs k en KNN")
plt.legend()
plt.show()

La gráfica muestra overfitting para k pequeños y underfitting para k muy grandes.

***e) Repita los experimentos anteriores utilizando la distancia de Minkowski para p ∈ {1, 2, 10}.***

In [ ]:
for p in [1, 2, 10]:
    knn = KNeighborsClassifier(n_neighbors=k_opt, p=p)  # p=1 (Manhattan), p=2 (Euclídea), p=10 (altamente no lineal)
    knn.fit(X_train, y_train)
    acc = knn.score(X_test, y_test)
    print(f"Accuracy con Minkowski p={p}: {acc:.4f}")

    No sé si va bien, no me ejecutaba antes

***f) ¿Cómo afecta el valor de p a los resultados? ¿Qué p cree que es mejor?***

Dados los resultados:

- p=1 (Manhattan): más robusto a outliers y diferencias grandes en un atributo.
- p=2 (Euclídea): la más usada; capta bien la noción de distancia “geométrica”.
- p=10: las mayores diferencias en una coordenada dominan la distancia → puede degradar rendimiento.

Normalmente p=2 (euclídea) es la mejor opción, aunque depende del dataset.

    Revisar esta respuesta, sacada de teoría

### **3. Selección de atributos. A continuaci ́on procederemos a la reducci ́on de la dimensión de los datos.**

---

***a) Usando el método VarianceThreshold de sklearn.feature selection para cierto umbral fijo u, elimine los atributos que no superen dicho umbral***

***b) Analice cómo afecta la selección de atributos al accuracy del modelo. Para ello, fijado un valor de k para KNN, calcule accuracy en test. El conjunto de entrenamiento tendr ́a los atributos seleccionados para un umbral u ∈ [0, 1] concreto. El resultado será una gráfica con el accuracy frente al umbral u***

***c) ¿Tienen sentido los casos u = 0 y u = 1?***

***d ) Ahora seleccione los mejores atributos del conjunto de datos utilizando SelectKBest de la librer ́ıa scikit-learn. Siga los siguientes pasos:***

***1. Importe el método SelectKBest***

***2. Utilice una estadística univariada como f_classif para la selección de atributos***

***3. Seleccione los mejores K atributos***

In [ ]:
from sklearn.feature_selection import SelectKBest

In [ ]:
from sklearn.feature_selection import f_classif

In [ ]:
selector = SelectKBest(score_func=f_classif, k=K)
selector.fit(X, y)
X_selected = selector.transform(X)

***e) Combinando el método de selección de atributos con el clasificador KNN para un valor de k fijo, determinte cuál es el mejor valor de K. Nótese que k hace referencia al número de vecinos en KNN y K es el número de atributos seleccionado***

***f ) A continuación implemente el método de selección de atributos mRMR. Para ello, complete el archivo mRMR.py.***

***g ) ¿Cuál es su mejor valor de k?***

***h) ¿Cul es el papel de la información mutua en el método mRMR? ¿Se podría sustituir por otra métrica?***

***i ) ¿Qué método de selección de atributos, de los dos utilizados, considera que es mejor?***